In [1]:
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA

# from openai import OpenAI
# import chromadb

# Loading the environment which contains the OPENAI_API key
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# # Loading the PDF file
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.pdf") # PDF file path


# # loader = PyPDFLoader(r"C:\Users\angad\OneDrive\Desktop\LangChain\india-national-building-code-nbc-2016-vol-1 (1) (1).pdf")
# pages = loader.load_and_split() # essentially splits the pdf into pages with the `page_content` as the text information and `metadata` as the source


In [3]:
# from langchain.document_loaders import TextLoader
# from langchain.chains import ConversationalRetrievalChain
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import VectorDBQA
# from langchain.chains import RetrievalQA
# # from openai import OpenAI
# # import chromadb

# embeddings = OpenAIEmbeddings() # creating the embeddings
# llm = ChatOpenAI() # the llm model 

# documents = pages

# # concatenating all the text in the pdf into one string
# text = ""
# for i in pages:
#     text += i.page_content

# # text splitter
# text_splitter = CharacterTextSplitter(
#         separator="\n",
#         chunk_size=1500,
#         chunk_overlap=500,
#         length_function=len,
#     )
# chunks = text_splitter.split_text(text)


In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter


loader = TextLoader("fire_life_safety.md")

# loader = TextLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.md")
md_text = loader.load()[0].page_content

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_text)

# Char-level splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size = 1000
# chunk_overlap = 500
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size, chunk_overlap=chunk_overlap
# )

# text splitter
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=2000,
        chunk_overlap=1000,
        length_function=len,
    )

# Split
splits = text_splitter.split_documents(md_header_splits)
splits
chunks = [i.page_content for i in splits]



In [5]:
# import pickle

# with open('vectorstore_bsc', 'rb') as file:
#     # Use pickle.dump() to serialize and save the object to the file
#     vectorstore = pickle.load(file)

In [6]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

# HuggingFace Embeddings
# embeddings = HuggingFaceInstructEmbeddings(
#     query_instruction="Represent the query for retrieval: "
# )
embeddings = OpenAIEmbeddings() # creating the embeddings
llm = ChatOpenAI(model = 'gpt-4') # the llm model 

vectorstore = FAISS.from_texts(chunks, embeddings) 

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4}) # setting up the text retriever. This allows us to choose the number of documents to be returned

In [8]:
# # GPT4All is a local GPT model, change the model to the location on your PC where the model is installed

# from langchain.llms import GPT4All

# llm = GPT4All(
#     model="/Users/rlm/Desktop/Code/gpt4all/models/nous-hermes-13b.ggmlv3.q4_0.bin", # change this with your path for GPT4ALL
#     max_tokens=2048, # max tokens to generate
# )

In [9]:
# Create retrieval QA chain
# from langchain.memory import ConversationSummaryMemory
# memory = ConversationSummaryMemory(llm=llm)
# from langchain.memory import ConversationBufferWindowMemory




# qa_chain = RetrievalQA.from_chain_type(
#     retriever=retriever,
#     llm=llm,
#     memory=ConversationBufferWindowMemory(k=5), # your preferred LLM 
# )

# Ask a question
# query = "what is the title?"
# result = qa_chain.run(query)
# print(result)

In [20]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT



from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

bot = ConversationalRetrievalChain.from_llm(
    llm, retriever, memory=memory, verbose=False
)

In [12]:
query = "Do the rules in my set share common underlying principles that can be flexibly combined to assess whether a model adheres to all of them?"
result = bot({"question": query})#
result['answer']

"The text doesn't provide enough information to definitively answer your question. Based on the provided text, the National Building Code of India outlines specific rules for fire and life safety in buildings and these rules must be adhered to. However, it is not clear if these rules can be flexibly combined based on common underlying principles to evaluate if a model complies with all of them. The text mentions that exceptions and deviations to the general provisions can be considered in case of practical difficulty or to avoid unnecessary hardship, without sacrificing reasonable safety. Yet, it doesn't specifically address flexible combination of rules based on common principles."

In [13]:
query = "What rules require calculating distance from one point to another in any capacity?"
result = bot({"question": query})#
result['answer']

'The rules that necessitate the calculation of distance from one point to another are:\n\n1) Rule 6.7.2.1: This rule requires calculating the distance from points within the area to the nearest outside walls where exit doors could be provided in buildings used for aircraft assembly or other occupancy requiring undivided floor areas. If the distances are in excess of 45 m, specific requirements for distance to exits must be satisfied. It also states that in no case may the distance of travel to reach the nearest exit exceed 65 m where smoke venting is required as a condition for permitting distances of travel to exits in excess of the maximum otherwise allowed.\n\n2) Rule 4.4.2.2: This rule requires calculating the travel distance on the floor to ensure it does not exceed the distance given in Table 5. The distance is measured from the most remote point within a storey or a mezzanine floor along the natural and unobstructed path to an exit.'

In [14]:
query = "What rules require calculating the total number of certain elements?"
result = bot({"question": query})#
result['answer']

'The context provided does not include any specific rules that necessitate the calculation of the total number of specific elements.'

In [15]:
query = "The other rules require what kind of stuff other than counting the total number of certain elements or calculating the distance?"
result = bot({"question": query})#
result['answer']

'The rules mentioned in the context provided also mandate the following requirements apart from counting the total number of certain elements or calculating distance:\n\n1. Rules govern the height for different types of buildings according to their occupancy, the width of streets providing approach to the building, permitted floor area ratio (FAR), and permitted open space around or inside buildings.\n\n2. Adequate passageway and clearances are required for firefighting vehicles.\n\n3. Attention must be paid to factors that limit fire spread when designing openings in fire-resistant walls and floors.\n\n4. A doorway or opening in a fire-resistant wall on any floor should not exceed specified dimensions. They also need to be protected with fire-resisting doors of a certain fire rating.\n\n5. Escalator openings must have provisions to avoid smoke spill and need to have sprinklers installed on all sides of the cut out opening on each floor.\n\n6. Certain systems are required for specific 

In [16]:
query = "List all the fundamental things such as calculating the distance or counting the numeber of elements etc which can encompass all rules"
result = bot({"question": query})#
result['answer']

'The basic principles common to all rules include:\n\n1. Calculation of distances: This involves measuring the travel distance from the most remote point within a storey or a mezzanine floor along the natural and unobstructed path of horizontal or vertical egress travel to the door to an exit. The travel distance on the floor should not exceed the distance given in the specific table (referenced as Table 5 in the text).\n\n2. Number and arrangement of exits: The minimum required number of exits in a building is determined based on occupant load and width required per person as appropriate to the type of exit for respective occupancies. Exits should be placed as remote from each other as possible and arranged to provide direct access in separate directions from any point in the area served.\n\n3. Capacity of exit access: The width of corridors, aisles or ramps required for exit access should be sufficient to ensure a smooth flow of occupants to the exit. The calculation of capacity of e

In [18]:
def split_string(input_string, max_length=180):
    # max_length = 125
    chunks = []

    current_chunk = ""
    for char in input_string:
        if char == '\n' or len(current_chunk) == max_length:
            chunks.append(current_chunk)
            current_chunk = ""
        else:
            current_chunk += char

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

large_string = result['answer']

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

The basic principles common to all rules include:

1. Calculation of distances: This involves measuring the travel distance from the most remote point within a storey or a mezzanine floor along the natural and unobstructed path of 
orizontal or vertical egress travel to the door to an exit. The travel distance on the floor should not exceed the distance given in the specific table (referenced as Table 5 in th
 text).

2. Number and arrangement of exits: The minimum required number of exits in a building is determined based on occupant load and width required per person as appropriate to the type
of exit for respective occupancies. Exits should be placed as remote from each other as possible and arranged to provide direct access in separate directions from any point in the 
rea served.

3. Capacity of exit access: The width of corridors, aisles or ramps required for exit access should be sufficient to ensure a smooth flow of occupants to the exit. The calculation 
f capacity of exit acc

In [19]:
query = 'are there any more?'
result = bot({"question": query})#
result['answer']

'The document does not provide any additional fundamental principles that can encompass all rules. However, it does mention that the purpose of this Part of the Code is to minimize danger to life and property from fire through occupancy-wise classification, constructional aspects, egress requirements and protection features. It also emphasizes the importance of maintaining all means of exit and fire protection measures during construction, repairs, alterations or additions to a building.'

In [21]:
query = 'List all fundamental principles that can encompass all rules'
result = bot({"question": query})#
result['answer']

'The document does not provide a specific list of fundamental principles that encompass all rules. However, from the given excerpts, we can infer several key principles that guide the rules for fire and life safety in the National Building Code of India:\n\n1. Fire Prevention: Buildings should be constructed and maintained in a manner that minimizes the risk of fire. This includes the use of non-flammable materials and the installation of fire suppression systems.\n\n2. Life Safety: Buildings must be designed in a way that ensures the safety of occupants in case of fire. This includes adequate means of egress, the provision of refuge areas, and the need for effective evacuation plans.\n\n3. Regular Updates: The rules and standards referred to in the code are subject to revision, encouraging parties to apply the most recent editions of the standards.\n\n4. Accessibility: Buildings must also be designed keeping in mind the needs of the elderly and persons with disabilities to ensure a sm

In [22]:
large_string = result['answer']

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

The document does not provide a specific list of fundamental principles that encompass all rules. However, from the given excerpts, we can infer several key principles that guide t
e rules for fire and life safety in the National Building Code of India:

1. Fire Prevention: Buildings should be constructed and maintained in a manner that minimizes the risk of fire. This includes the use of non-flammable materials and the installatio
 of fire suppression systems.

2. Life Safety: Buildings must be designed in a way that ensures the safety of occupants in case of fire. This includes adequate means of egress, the provision of refuge areas, and
the need for effective evacuation plans.

3. Regular Updates: The rules and standards referred to in the code are subject to revision, encouraging parties to apply the most recent editions of the standards.

4. Accessibility: Buildings must also be designed keeping in mind the needs of the elderly and persons with disabilities to ensure a smooth and 

In [ ]:
from chromadb.utils import embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name="text-embedding-ada-002"
            )

In [ ]:
client = chromadb.Client()
collection = client.get_or_create_collection("angad-msc",embedding_function=openai_ef)

In [ ]:
# Adding the text and metadata to chromadb
documents = chunks
metadatas = [{"source":"angad_msc"} for i in range(len(chunks)) ]
ids = [str(i) for i in range(len(chunks))]
collection.add(
            documents = documents,
            metadatas = metadatas,
            ids = ids
    )

In [ ]:
# question = "Who is my supervisor?"
question = "summarise section 3"
results = collection.query(query_texts = [question], n_results = 5)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

results = collection.query(query_texts = [question], n_results = 2)

template = """The following piece of text is given: {text}. Please answer any following questions ONLY using THIS piece of text in a brief manner."""
human_template = "{question}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() 
context = ''
for i in range(len(results['documents'])):
    context = context + results['documents'][0][i] + '\n'
chain.invoke({"text": context, "question": question})


In [ ]:
llm = ChatOpenAI()
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=True
    )